# 1. Conectarse a la base de datos de Sophia2

In [ ]:
import mysql.connector

In [ ]:
USER="sophia2api"
HOST="45.56.109.120"
PORT="14096"
PASSWORD="frmv9485"
DATABASE="Sun"

In [ ]:
db_connection = mysql.connector.connect(user=USER,host=HOST,password=PASSWORD,database=DATABASE, port=PORT)

# 2. Explorar la base de datos de Sophia2

In [ ]:
import pandas as pd
#from pandasql import sqldf

In [ ]:
QUERY = 'select * from news where country="chile" and date >= "2021-01-01" and date <= "2021-01-31" ORDER BY RAND() limit 1000;'

In [ ]:
table_news=pd.read_sql(QUERY,con=db_connection)

In [ ]:
table_news

# 3. Guardar el dataset en un CSV

In [ ]:
DATASET_FILE = "../datasets/chile_1000rand_01_2021.csv"

In [ ]:
table_news.to_csv(DATASET_FILE, index=False, encoding="utf-8")

# 4. Cargar un dataset en un dataframe Pandas

In [ ]:
df = pd.read_csv(DATASET_FILE)

In [ ]:
df[:10]

# 5. Explorar el dataset utilizando SQL

In [ ]:
from pandasql import sqldf

- ¿Cuáles son los medios chilenos indexados?

In [ ]:
query_sql="SELECT distinct media_outlet FROM df;"
df_result=sqldf(query_sql)
df_result

- ¿Cuáles es la cantidad de noticias por medios de prensa?

In [ ]:
query_sql="SELECT media_outlet, count(*) FROM df GROUP BY media_outlet ORDER BY count(*) DESC;"
df_result=sqldf(query_sql)
df_result

- ¿Cuáles es la cantidad de noticias por día?

In [ ]:
query_sql="SELECT date, count(*) FROM df GROUP BY media_outlet ORDER BY date ASC;"
df_result=sqldf(query_sql)
df_result

- ¿Qué forma tienen las URLs de algún medio particular?

In [ ]:
#cambiar el número de caracteres en las columnas de un dataframe (-1 muestra todo, pero no es recomendado)
pd.set_option('display.max_colwidth', 150)

In [ ]:
query_sql="SELECT url FROM df WHERE media_outlet='latercera' LIMIT 30;"
df_result=sqldf(query_sql)
df_result

In [ ]:
pd.set_option('display.max_colwidth', 80)

# 6. Preprocesar los títulos con spaCy

In [ ]:
query_sql="SELECT title FROM df;"
df_result=sqldf(query_sql)
df_result

In [ ]:
df_result['title'][1]

In [ ]:
for index, row in df_result.iterrows():
    print(df_result['title'][index])

In [ ]:
import spacy

nlp = spacy.load("es_core_news_md")
print(spacy.__version__)

- Contar los sustantivos más frecuentes en los titulos

In [ ]:
wordfreq = {}

for index, row in df_result.iterrows():
    title = df_result['title'][index]
    doc = nlp(title)
    
    for token in doc:
        word=token.text
        if token.pos_ == "NOUN":
            if word not in wordfreq:
                wordfreq[word] = 0 
            wordfreq[word] += 1

In [ ]:
wordfreq

In [ ]:
import operator
sorted_wordfreq = dict( sorted(wordfreq.items(), key=operator.itemgetter(1),reverse=True))

In [ ]:
sorted_wordfreq

- Contar los verbos más frecuentes en los titulos

In [ ]:
wordfreq = {}

for index, row in df_result.iterrows():
    title = df_result['title'][index]
    doc = nlp(title)
    
    for token in doc:
        word=token.text
        if token.pos_ == "VER":
            if word not in wordfreq:
                wordfreq[word] = 0 
            wordfreq[word] += 1


sorted_wordfreq = dict( sorted(wordfreq.items(), key=operator.itemgetter(1),reverse=True))

sorted_wordfreq

- Contar las palabras más frecuentes en los titulos

In [ ]:
wordfreq = {}

for index, row in df_result.iterrows():
    title = df_result['title'][index]
    doc = nlp(title)
    
    for token in doc:
        word=token.text

        if word not in wordfreq:
            wordfreq[word] = 0 
        wordfreq[word] += 1


sorted_wordfreq = dict( sorted(wordfreq.items(), key=operator.itemgetter(1),reverse=True))

sorted_wordfreq

In [ ]:
# TOP - 100
sorted_wordfreq_top100 = {k: sorted_wordfreq[k] for k in list(sorted_wordfreq)[:100]}

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.pyplot import figure

figure(figsize=(16, 12), dpi=80)

pos = np.arange(len(sorted_wordfreq_top100.keys()))
width = 1.0     # gives histogram aspect to the bar diagram

ax = plt.axes()
ax.set_xticks(pos + (width / 2))
ax.set_xticklabels(sorted_wordfreq_top100.keys(),rotation='vertical')

plt.bar(sorted_wordfreq_top100.keys(), sorted_wordfreq_top100.values(), width, color='g')

plt.show()